# Mathematical Formulation
Suppose the following variables and constants:
* Suppose $n$ as the number of staff on the trip.  
* Let $g$ be the fuel cost of our van, with the unit of RMB/km. $c_h$ as the hotel room cost for one person per night. $c_s$ as the salary cost for one person per day. $c_v$ as the cost for renting a van per day.  
* Let $D$ be the distance matrix. Each entry ($d_{ij}$) is the most recommended route suggested by Baidu Map. Note that $D$ is not necessarily symmetric as the map may suggest different routes when starting city and destination is changed.  
* Let $V$ be the set of indices that represent all the cities.  
* Suppose binary variable $x_{ij}$ be 1 if route from city $i$ to $j$ is used, 0 otherwise. And $i,j \in V$.  
* Suppose matrix $T$ to have its $ij$-entry ($t_{ij}$) as binary variables indicating whether the estimated time using route from city $i$ to city $j$ exceed 8 hours.  
* Also, create $u_i$ for $i\in V$ where $u_i = $ the order of visit to city $i$.  

Using the above variables and constants, we write the following linear optimization problem:  
$\begin{array}{ccc}
\text{minimize} & \sum_{i,j} [t_{ij}\cdot n\cdot(c_s + c_h) + g\cdot d_{ij} + (t_{ij}+1)\cdot c_v]x_{ij} & \\
\text{subject to} & \sum_{j,i\neq j} x_{ij} = 1 & \forall i \\
 & \sum_{i,i\neq j} x_{ij} = 1 & \forall j \\
 & u_1 = 1 & \\
 & 2 \leq u_i \leq n & \forall 2\leq i \leq n \\
 & u_i - u_j + nx_{ij} \leq n - 1 & \forall 2\leq i,j \leq n 
\end{array}$  

Below is the code:

In [1]:
def buildVanModel(n, g, ch, cs, cv, D, T):
    from pyscipopt import Model, quicksum
    modelVan = Model("Travelling Van")
    
    N = len(D)
    x = {}
    u = {}
    
    for i in range(1, N+1):
        u[i] = modelVan.addVar(lb = 1, ub = N, vtype = "I", name = "u(%s)"%i)
        for j in range(1, N+1):
            if i != j:
                x[i,j] = modelVan.addVar(vtype = "B", name = "x(%s, %s)"%(i, j))
            
    for i in range(1, N+1):
        modelVan.addCons(quicksum(x[i,j] for j in range(1, N+1) if i != j) == 1, "Only 1 out at %s"%i)
        modelVan.addCons(quicksum(x[j,i] for j in range(1, N+1) if i != j) == 1, "Only 1 in at %s"%i)
        
    modelVan.addCons(u[1] == 1, "u(1) = 1")
    for i in range(2, N+1):
        modelVan.addCons(2 <= (u[i] <= N), "Bound for u(%s)"%i)
    
    for i in range(2, N+1):
        for j in range(2, N+1):
            if i != j:
                modelVan.addCons(u[i] - u[j] + N*x[i,j] <= N-1, "Only one loop in graph")
                
    modelVan.setObjective(quicksum((T[i][j-1]*n*(cs+ch) + n*cs + g*D[i][j-1] + (T[i][j-1]+1)*cv)*x[i,j] for (i,j) in x), 
                          "minimize")
    modelVan.data = x, u
    
    return modelVan

In [2]:
n = 2.0
g = 0.8
ch = 180.0
cs = 120.0
cv = 780.0
Cities = {1:'Jinan', 2:'Jiyangqu', 3:'Yucheng', 4:'Gaotangxian', 5:'Liaocheng', 6:'Yangguxian', 7:'Taian', 
           8:'Xintai', 9:'Zibo', 10:'Binzhou'}
D = {1: [0, 46, 66.7, 102.6, 125.6, 176.8, 74.8, 132.6, 111, 153.3], 
     2: [45.5, 0, 74.1, 107.6, 135, 192.4, 126.4, 155.3, 110.2, 109], 
     3: [67.6, 71.3, 0, 41.2, 97.8, 155.2, 109.7, 189.9, 152.1, 176.4], 
     4: [108.7, 104.9, 41.2, 0, 54.2, 112.1, 146.1, 230.7, 191, 215.4], 
     5: [125.5, 134.4, 93.5, 54, 0, 45.9, 130.5, 201.4, 215.2, 239.5], 
     6: [183, 191.8, 155, 111.8, 45.9, 0, 153.2, 224, 270.1, 296.9], 
     7: [75, 122.8, 119, 172, 130.8, 153.2, 0, 82.8, 156.9, 230], 
     8: [126.8, 161.8, 189.4, 229.9, 201.7, 223.4, 81.7, 0, 130, 182.3], 
     9: [112, 119.7, 152.3, 191.2, 211.1, 268.6, 159.2, 126.8, 0, 85.3], 
     10: [160.5, 99.4, 170.9, 212, 237.8, 295.3, 217.3, 184.3, 80.7, 0]}
T = {1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     6: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     7: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     8: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
     10: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [3]:
def reportOnVan(n, g, ch, cs, cv, D, T):
    modelVan = buildVanModel(n, g, ch, cs, cv, D, T)
    modelVan.optimize()
    x, u = modelVan.data
    print("Minimized cost of transportation: " + str(modelVan.getObjVal()) + " RMB")
    print()
    
    print("The cities and their visiting order: ")
    for i in range(1, len(D)+1):
        print(Cities[i] + ": " + str(modelVan.getVal(u[i])))

In [4]:
reportOnVan(n, g, ch, cs, cv, D, T)

Minimized cost of transportation: 10843.36 RMB

The cities and their visiting order: 
Jinan: 1.0
Jiyangqu: 10.0
Yucheng: 2.0
Gaotangxian: 3.0
Liaocheng: 4.0
Yangguxian: 5.0
Taian: 6.0
Xintai: 7.0
Zibo: 8.0
Binzhou: 9.0


Jinan: 1.0
Jiyangqu: 10.0
Yucheng: 2.0
Gaotangxian: 3.0
Liaocheng: 4.0
Yangguxian: 5.0
Taian: 6.0
Xintai: 7.0
Zibo: 8.0
Binzhou: 9.0


NameError: name 'modelVan' is not defined